In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import string
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/fake-news-detection/True.csv
/kaggle/input/fake-news-detection/Fake.csv


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#let's see the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
df_fake = pd.read_csv("/kaggle/input/fake-news-detection/Fake.csv")
df_true = pd.read_csv("/kaggle/input/fake-news-detection/True.csv")

In [5]:
# Add a label column to indicate fake (1) or true (0)
df_fake['label'] = 1
df_true['label'] = 0

# Concatenate the DataFrames to merge the datasets
merged_df = pd.concat([df_fake, df_true], ignore_index=True)

# Shuffle the merged DataFrame to randomize the order
merged_df = merged_df.sample(frac=1).reset_index(drop=True)


In [6]:
merged_df.head(10)

,title,text,subject,date,label
0,PRESIDENT TRUMP BLASTS OBAMA: “Obamacare Was a...,President Trump gathered the Senators in the S...,politics,"Jul 19, 2017",1
1,Keith Olbermann: Donald Trump Wants To Protec...,"Four years ago, many complained about the fact...",News,"October 3, 2016",1
2,Iraq court rules no region can secede after Ku...,BAGHDAD (Reuters) - Iraq s Supreme Federal Cou...,worldnews,"November 6, 2017",0
3,Church Installs ‘Watering System’ To Get Rid ...,Anyone who has read the Bible knows that that ...,News,"February 20, 2016",1
4,When The View’s WHOOPI GOLDBERG Told Hillary W...,The sympathetic (and borderline communist) wom...,politics,"Sep 14, 2017",1
5,CLASSIC VIDEO: Angry Woman Nails It Describing...,This woman nails it! LANGUAGE ALERT!,Government News,"Jan 27, 2016",1
6,JOE BIDEN’S NIECE Too Big To Jail? Released Af...,"The rules must not apply to Caroline Biden, ni...",politics,"Jun 10, 2017",1
7,Trump to announce transportation secretary pic...,WASHINGTON (Reuters) - President-elect Donald ...,politicsNews,"November 29, 2016",0
8,Brexit talks cannot move to phase 2 without bo...,DUBLIN (Reuters) - Britain s talks on exiting ...,worldnews,"October 27, 2017",0
9,EU-U.S. trade deal in doubt as France urges en...,PARIS/BRUSSELS (Reuters) - France cast serious...,politicsNews,"August 30, 2016",0


In [7]:
merged_df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [8]:
merged_df.shape

(44898, 5)

In [9]:
merged_df['Content'] = merged_df['subject'] + ' : ' + merged_df['title']

In [10]:
merged_df

,title,text,subject,date,label,Content
0,PRESIDENT TRUMP BLASTS OBAMA: “Obamacare Was a...,President Trump gathered the Senators in the S...,politics,"Jul 19, 2017",1,politics : PRESIDENT TRUMP BLASTS OBAMA: “Obam...
1,Keith Olbermann: Donald Trump Wants To Protec...,"Four years ago, many complained about the fact...",News,"October 3, 2016",1,News : Keith Olbermann: Donald Trump Wants To...
2,Iraq court rules no region can secede after Ku...,BAGHDAD (Reuters) - Iraq s Supreme Federal Cou...,worldnews,"November 6, 2017",0,worldnews : Iraq court rules no region can sec...
3,Church Installs ‘Watering System’ To Get Rid ...,Anyone who has read the Bible knows that that ...,News,"February 20, 2016",1,News : Church Installs ‘Watering System’ To G...
4,When The View’s WHOOPI GOLDBERG Told Hillary W...,The sympathetic (and borderline communist) wom...,politics,"Sep 14, 2017",1,politics : When The View’s WHOOPI GOLDBERG Tol...
...,...,...,...,...,...,...
44893,BAD NEWS FOR THE SPEAKER: Many Republicans Agr...,Trump knows the art of the deal like no one el...,politics,"Sep 8, 2017",1,politics : BAD NEWS FOR THE SPEAKER: Many Repu...
44894,Supporters of Egypt presidential hopeful arres...,CAIRO (Reuters) - Egyptian security forces hav...,worldnews,"December 13, 2017",0,worldnews : Supporters of Egypt presidential h...
44895,Britain's government to push ahead with plan o...,LONDON (Reuters) - Britain s Prime Minister Th...,worldnews,"November 17, 2017",0,worldnews : Britain's government to push ahead...
44896,Biafra separatists sponsored by Nigerian gover...,ABUJA (Reuters) - A Nigerian minister accused ...,worldnews,"September 18, 2017",0,worldnews : Biafra separatists sponsored by Ni...


In [11]:
X = merged_df.drop(columns='label',axis=1)
Y = merged_df['label']


In [12]:
#Un comment the below line of code to see the X and Y
# print(X)
# print(Y)

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(Content):
    stemmed_con = re.sub('[^A-Za-z]',' ',Content)
    stemmed_con = stemmed_con.lower()
    stemmed_con = stemmed_con.split()
    stemmed_con = [port_stem.stem(word) for word in stemmed_con if not word in stopwords.words('english')]
    stemmed_con = ' '.join(stemmed_con)
    return stemmed_con

In [15]:
merged_df['Content'] = merged_df['Content'].apply(stemming)

In [16]:
merged_df['Content']

0        polit presid trump blast obama obamacar big bi...
1        news keith olbermann donald trump want protect...
2        worldnew iraq court rule region seced kurdish ...
3        news church instal water system get rid homele...
4        polit view whoopi goldberg told hillari lost t...
                               ...                        
44893    polit bad news speaker mani republican agre tr...
44894    worldnew support egypt presidenti hope arrest ...
44895    worldnew britain govern push ahead plan specif...
44896    worldnew biafra separatist sponsor nigerian go...
44897    news hypocrit jeff session word perjuri came b...
Name: Content, Length: 44898, dtype: object

In [17]:
#the below code will separate Content and label column and save it in X and Y
X = merged_df['Content'].values
Y = merged_df['label'].values

In [18]:
X

array(['polit presid trump blast obama obamacar big big lie lie lie presid said time video',
       'news keith olbermann donald trump want protect right poison dog video',
       'worldnew iraq court rule region seced kurdish independ bid', ...,
       'worldnew britain govern push ahead plan specif brexit date',
       'worldnew biafra separatist sponsor nigerian govern oppon minist',
       'news hypocrit jeff session word perjuri came back bite ass'],
      dtype=object)

In [19]:
# Converting textual data to numerical data
vector = TfidfVectorizer()
vector.fit(X)

TfidfVectorizer()

In [20]:
transform_X = vector.transform(X)

In [21]:
tfidf_array = transform_X.toarray()

print(tfidf_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
type(tfidf_array)

numpy.ndarray

In [23]:
# Splitting data into train and test

X_train,X_test,Y_train,Y_test = train_test_split(tfidf_array, Y, test_size=0.2, stratify=Y, random_state=42)

In [24]:
model = LogisticRegression()

In [25]:
model.fit(X_train,Y_train)

LogisticRegression()

In [26]:
model.predict(X_test)

array([1, 0, 0, ..., 1, 1, 0])

In [27]:
model.score(X_test,Y_test)

1.0